In [19]:
import gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange
from IPython import display

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

In [71]:
class ConvStack(nn.Module):
    """
    Implements the conv_stack module as described in figure 6
    """
    def __init__(self, k1, c1, k2, c2, k3, c3):
        super(ConvStack, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=c1, kernel_size=k1, padding=(k1-1)//2)
        self.conv2 = nn.Conv2d(in_channels=c1, out_channels=c2, kernel_size=k2, padding=(k2-1)//2)
        self.conv3 = nn.Conv2d(in_channels=c2, out_channels=c3, kernel_size=k3, padding=(k3-1)//2)
        
    def forward(self, X):
        if not torch.is_tensor(X):
            X = torch.from_numpy(X).type(torch.FloatTensor)
            
        conv1_X = self.conv1(X)
        conv1_relu = F.relu(conv1_X)
        
        conv2_X = self.conv2(conv1_X)
        conv2_relu = F.relu(conv2_X + conv1_relu)
        
        conv3_X = self.conv3(conv2_relu)
        
        return conv3_X
    
    
class ResConv(nn.Module):
    """
    Implements the res_conv module as described in figure 7 in the paper
    """
    def __init__(self, use_extra_convolution=True):
        super(ResConv, self).__init__()
        self.use_extra_convolution = use_extra_convolution
        self.extra_conv = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=1)
        
        if use_extra_convolution:
            conv1_in_channels = 64
        else:
            conv1_in_channels = 3
        
        self.conv1 = nn.Conv2d(in_channels=conv1_in_channels, out_channels=32, kernel_size=3, padding=(3-1)//2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=(5-1)//2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=(3-1)//2)
        
    def forward(self, X):
        if not torch.is_tensor(X):
            X = torch.from_numpy(X).type(torch.FloatTensor)
        if self.use_extra_convolution:
            c = self.extra_conv(X)
        else:
            c = X
        
        conv1_relu = F.relu(self.conv1(c))
        conv2_relu = F.relu(self.conv2(conv1_relu))
        rc3 = self.conv3(conv2_relu)
        rc = c + rc3
        return rc
    

class StateTransitionModule(nn.Module):
    """
    Implements the state transition function g(s,z,a) as described in figure 9 in the paper
    An action, a state and a latent variable at time t-1 is transitioned to 
    a state at time t.
    """
    def __init__(self):
        super(StateTransitionModule, self).__init__()
        self.res_conv1 = ResConv()
        self.res_conv2 = ResConv()
        
    def pool_inject(self, X):
        """
        Implements the pool & inject module as described in figure 8 in the paper
        """
        if not torch.is_tensor(X):
            X = torch.from_numpy(X).type(torch.FloatTensor)

        height, width = X.shape[2:]
        m = F.max_pool2d(X, kernel_size=(height, width), stride=(1, 1))
        tiled = m.repeat(1, 1, height, width)
        pi = torch.cat([tiled, X], axis=1) # concat on the Color channel
        return pi
    
    def forward(self, a, s, z):
        if z is None:
            c = torch.cat([a, s], axis=1) # concat on the Color channel
        else:
            c = torch.cat([a, s, z], axis=1) # concat on the Color channel
            
        rc1_relu = F.relu(self.res_conv1(c))
        pi = self.pool_inject(rc1_relu)
        s_next = self.res_conv2(pi)
        
        return s_next

In [70]:
s = np.flip(np.transpose(state, axes=[2, 0, 1])).copy()
res_conv = ResConv() 
pool_inject(np.array([s])).shape

torch.Size([1, 3, 96, 96]) torch.Size([1, 3, 96, 96])


torch.Size([1, 6, 96, 96])